# Assignment 1 – Backprop

In [2]:
#@title Library Imports [do not change]

import importlib 
!git clone https://www.github.com/rycolab/intro-nlp-f22.git
utils = importlib.import_module("intro-nlp-f22.assignment_1.utils")

import re
import random
from collections import defaultdict
import itertools
from abc import ABC, abstractmethod
import math

Cloning into 'intro-nlp-f22'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 32 (delta 6), reused 30 (delta 4), pack-reused 0
Unpacking objects: 100% (32/32), done.


In [32]:
#@title Select and Parse Math Problems [do not change]

#@markdown select math problem

math_problem_i = "3" #@param [0,1,2,3]
math_problem = utils.MATH_PROBLEMS[int(math_problem_i)]
print(math_problem)

parser = utils.Parser()
infix, in_vars = parser.parse(math_problem["problem"], in_vars = math_problem["in_vars"])
print(infix, in_vars)

{'problem': 'z + sin(x^(2) + (y * exp(z)))', 'in_vars': {'x': 2.0, 'y': -1.0, 'z': 0.0}, 'output': 0.14, 'derivative': {'x': -3.96, 'y': -0.99, 'z': 1.99}}
['z', '+', ['sin', [['x', '^', 2], '+', ['y', '*', ['exp', 'z']]]]] {'z': 0.0, 'x': 2.0, 'y': -1.0}


In [36]:
from ast import Not
#@title ToDo1: Building

class Builder():

    class Node():
        """
          If node is a constant then var=None
          if node is leaf then parents=[]
        """
        def __init__(self, var, parents=[], operator: str=None, 
                    value: float=None, is_const: bool=False):
            self.operator = operator
            self.var = var
            self.value = value
            self.is_const = is_const

            self.parents = parents
            self.children = []
            for p in self.parents:
                p.children.append(self)

            # Holds the gradient, gradient[var]=partial wrt 'var'.
            self.grad = {}
        
        def __str__(self):
            if self.parents is None:
              self.parents = []
            return f'var: {str(self.var):6} op: {str(self.operator):6} val: {self.value} hash: {hash(self)} len: {len(self.children)}'


    def build_graph(self, infix, in_vars: dict, avail_vars: set()):
        """
          Builds the computational graph recursively.
          Values are not initiated here.
          Operators are strings, i.e not mapped to respective Operator class.
        """
        if not isinstance(infix, list):
            try:
                n = self.Node(None, value=float(infix), is_const=True)
            except ValueError:
                if not (infix in in_vars):
                    print(f'Variable \'{infix}\' is used but is not in \'in_vars\'.')
                # We must check that variable Node doesnt already exist!
                # e.g. if input is x+x
                # must be able to get a node from knowing its variable name
                # i.e mapping from variable name to Node object.
                if infix in self.in_nodes:
                    n = self.in_nodes[infix]
                else:
                    n = self.Node(infix)
                    self.in_nodes[infix] = n
            return n
          
        if len(infix) == 3:
            p1 = self.build_graph(infix[0], in_vars, avail_vars)
            p2 = self.build_graph(infix[2], in_vars, avail_vars)
            n = self.Node(avail_vars.pop(), [p1, p2],  operator=infix[1])
        elif len(infix) == 2:
            p = self.build_graph(infix[1], in_vars, avail_vars)
            n = self.Node(avail_vars.pop(), [p], operator=infix[0])
        else:
            raise NotImplementedError('Only unary and binary ops are supported')
        return n
    
    def topo_sort(self, node):
        """
            Return the computational graph in topological order.
        """
        sorted = []
        visited = set()

        def dfs_topo(node):
            """ Topo sort from node that can reach all others. """
            for p in node.parents:
                if p not in visited:
                    visited.add(p)
                    dfs_topo(p)
            sorted.append(node)         

        dfs_topo(node)
        return sorted

    def __init__(self, infix: list, in_vars: dict = {}):
        """
        infix: list of infix notation parse, e.g. [['exp', 2], '-', 3]
        in_vars: dict of input variables to ensure they are not used as intermediate or output variables
        RETURN: computation graph in a data structure of your choosing
        """

        ## some alphabetical vars to use as intermediate and output variables minus the input vars to avoid duplicates
        avail_vars = list(map(chr, range(97, 123))) + list(map(chr, range(945, 969)))
        if len(in_vars.keys()) > 0:
            avail_vars = set(avail_vars) - set(in_vars)
        self.avail_vars = sorted(list(set(avail_vars)), reverse=True)
        self.infix = infix
        self.in_nodes = {}
        print(infix)

        g = self.build_graph(infix, in_vars, avail_vars)
        self.graph = self.topo_sort(g)

        for n in self.graph:
            print(n)


if __name__ == '__main__':
    print(infix)
    g = Builder(infix, in_vars)
   # print(g.graph)
    #g = Builder([['x', '+', 'y'],'+',['z', '+', 'z2']], in_vars)
    
    

['z', '+', ['sin', [['x', '^', 2], '+', ['y', '*', ['exp', 'z']]]]]
['z', '+', ['sin', [['x', '^', 2], '+', ['y', '*', ['exp', 'z']]]]]
var: z      op: None   val: None hash: 8729743083705 len: 2
var: x      op: None   val: None hash: 8729743083717 len: 1
var: None   op: None   val: 2.0 hash: 8729743083721 len: 1
var: p      op: ^      val: None hash: 8729743083713 len: 1
var: y      op: None   val: None hash: 8729743083685 len: 1
var: σ      op: exp    val: None hash: 8729743083725 len: 1
var: u      op: *      val: None hash: 8729743083729 len: 1
var: η      op: +      val: None hash: 8729743083733 len: 1
var: ε      op: sin    val: None hash: 8729743083737 len: 1
var: t      op: +      val: None hash: 8729743083741 len: 0


In [5]:
#@title ToDo2: Operations

class Operator(ABC):

    @abstractmethod
    def f(self, a, b = None) -> float:
        raise NotImplementedError()
        return f_res

    @abstractmethod
    def df(self, a, b = None) -> list:
        raise NotImplementedError()
        return [df_res]

class Exp(Operator):

    def f(self, a, b = None):
        return math.exp(a)

    def df(self, a, b = None):
        return [math.exp(a)]

class Log(Operator):
    ## natural logarithm

    def f(self, a, b = None):
        return math.log(a)

    def df(self, a, b = None):
        return [1.0/a]

class Mult(Operator):

    def f(self, a, b):
        return a * b

    def df(self, a, b=None):
        return [b, a]

class Div(Operator):

    def f(self, a, b): # i did this
        return a / b

    def df(self, a, b): # i did this
        return [1.0 / b, -a / b**2]

class Add(Operator):

    def f(self, a, b): # i did this
        return a + b

    def df(self, a, b = None):
        return [1, 1]

class Sub(Operator):

    def f(self, a, b = None): # i did this
        if b is None:
            return -a # should be in multiply operator instead?
        return a-b

    def df(self, a, b = None):
        if b is None:
            return [-1]
        return [1,-1]

class Pow(Operator):

    def f(self, a, b):
        return a**b

    def df(self, a, b):
        if a <= 0: ## work-around: treat as unary operation if -a^b
            return [b * (a ** (b - 1))]
        else:
            return [b * (a ** (b - 1)), (a ** b) * math.log(a)]

class Sin(Operator):

    def f(self, a, b=None): # i did this
        return math.sin(a)

    def df(self, a, b=None):
        return [math.cos(a)]

class Cos(Operator):

    def f(self, a, b=None):
        return math.cos(a)

    def df(self, a, b=None):
        return [-math.sin(a)]

In [34]:
#@title ToDo 3: Executing

class Executor():

    def __init__(self, graph: dict, in_vars: dict = {}):
        """
        graph: computation graph in a data structure of your choosing
        in_vars: dict of input variables, e.g. {"x": 2.0, "y": -1.0}
        """
        self.graph = graph
        self.in_vars = in_vars
        self.fn_map = {"log": Log(), "exp": Exp(), "+": Add(), "-": Sub(), "^": Pow(), "sin": Sin(), "cos": Cos(), "*": Mult(), "/": Div()}
        self.output = -1
        self.derivative = {}

    ## forward execution____________________________

    def forward(self, ):
        for n in self.graph:
            if n.var in self.in_vars:
                n.value = self.in_vars[n.var]
            elif n.operator:
                inputs = [p.value for p in n.parents]
                assert len(inputs) == 1 or len(inputs) == 2
                n.value = self.fn_map[n.operator].f(*inputs)

        self.output = self.graph[-1].value

    ## backward execution____________________________

    def backward(self, ):
        for n in self.graph[::-1]:

            if n.operator:
                inputs = [p.value for p in n.parents]
                assert len(inputs) == 1 or len(inputs) == 2
                # check if one of the inputs are constant?
                # that may affect derivative?
                grad = self.fn_map[n.operator].df(*inputs)
                print(f'grad: {grad}')
                # Can it happen that two inputs have gradient with len 1.
                # yes? if one of the inputs are constant?
                for p in n.parents:
                    if p.is_const:
                        continue
                    if p in n.grad:
                        n.grad[p].append(grad.pop(0))
                    else:
                        n.grad[p] = [grad.pop(0)]

            # want to multiply all grads in this node, with the n.child.grad[n]
            if n.var:
                grads = [c.grad for c in n.children]
                print(grads)
                #[{'node1': [2,3]}, {'node2':[1], 'node3':[4]}]
                if n.children:
                    child_grad = sum([sum(g[n]) for g in grads])
                    for np in n.grad:
                        n.grad[np] = [child_grad * g for g in n.grad[np]]
                    if n.var in self.in_vars:
                        print(f'set gradient: {n.var}={child_grad}')
                        n.grad[n] = [child_grad]
                

            if n.var in self.in_vars:
                if len(n.grad[n]) != 1:
                    print('should not happen, grad of wrong length!')
                self.derivative[n.var] = n.grad[n][0]



if __name__ == '__main__':
  e = Executor(g.graph, in_vars=in_vars)
  e.forward()
  e.backward()
  print(e.output)
  print(e.derivative)

grad: [-1.0, -2.0]
[]
[{<__main__.Builder.Node object at 0x7f08d5617850>: [1.0, 1.0, 1.0, -1.0], <__main__.Builder.Node object at 0x7f08d3e79210>: [-1.0, -1.0, -1.0, -2.0]}]
set gradient: y=-5.0
[{<__main__.Builder.Node object at 0x7f08d5617850>: [1.0, 1.0, 1.0, -1.0], <__main__.Builder.Node object at 0x7f08d3e79210>: [-1.0, -1.0, -1.0, -2.0]}]
set gradient: x=2.0
-2.0
{'y': -5.0, 'x': 2.0}


In [37]:
#@title Test Function for Debugging [do not change]

utils.test_backprop(Builder, Executor, math_problem)
#my_test = {'problem': '(x/x+x)+(x^2+y)', 'in_vars': {'x': 5.0, 'y':1.0}, 'output': 32.0, 'derivative': {'x': 11.0, 'y':1.0}}
#my_test = {'problem': '(-x)^3', 'in_vars': {'x': 5.0}, 'output': -125.0, 'derivative': {'x': -75.0}}

#utils.test_backprop(Builder, Executor, my_test)

['z', '+', ['sin', [['x', '^', 2], '+', ['y', '*', ['exp', 'z']]]]]
var: z      op: None   val: None hash: 8729743104997 len: 2
var: x      op: None   val: None hash: 8729743104937 len: 1
var: None   op: None   val: 2.0 hash: 8729743104805 len: 1
var: p      op: ^      val: None hash: 8729743104789 len: 1
var: y      op: None   val: None hash: 8729743105013 len: 1
var: σ      op: exp    val: None hash: 8729743104797 len: 1
var: u      op: *      val: None hash: 8729743079445 len: 1
var: η      op: +      val: None hash: 8729743079573 len: 1
var: ε      op: sin    val: None hash: 8729743079505 len: 1
var: t      op: +      val: None hash: 8729743078185 len: 0
grad: [1, 1]
[]
grad: [-0.9899924966004454]
[{<__main__.Builder.Node object at 0x7f08d3c9be50>: [1], <__main__.Builder.Node object at 0x7f08d3c38510>: [1]}]
grad: [1, 1]
[{<__main__.Builder.Node object at 0x7f08d3c38950>: [-0.9899924966004454]}]
grad: [1.0, -1.0]
[{<__main__.Builder.Node object at 0x7f08d3c9b150>: [-0.9899924966004

In [38]:
#@title Test Function for Grading [do not change]

utils.test_backprop(Builder, Executor)

['x', '/', 'y']
var: x      op: None   val: None hash: 8729743104977 len: 1
var: y      op: None   val: None hash: 8729743104945 len: 1
var: p      op: /      val: None hash: 8729743105009 len: 0
grad: [1.0, -1.0]
[]
[{<__main__.Builder.Node object at 0x7f08d3c9bd10>: [1.0], <__main__.Builder.Node object at 0x7f08d3c9bb10>: [-1.0]}]
set gradient: y=-1.0
[{<__main__.Builder.Node object at 0x7f08d3c9bd10>: [1.0], <__main__.Builder.Node object at 0x7f08d3c9bb10>: [-1.0]}]
set gradient: x=1.0

0: problem: x/y, in_vars: {'x': 1.0, 'y': 1.0}
SUCCESS output: 1.0
SUCCESS derivative: {'y': -1.0, 'x': 1.0}
[['exp', 'x'], '-', ['y', '*', 2]]
var: x      op: None   val: None hash: 8729743078357 len: 1
var: p      op: exp    val: None hash: 8729743078309 len: 1
var: y      op: None   val: None hash: 8729743078313 len: 1
var: None   op: None   val: 2.0 hash: 8729743078389 len: 1
var: z      op: *      val: None hash: 8729743078149 len: 1
var: σ      op: -      val: None hash: 8729743078209 len: 0
gr